In [1]:
from papers import *

# imports
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException

In [18]:
def ads_get_ref_cit(url, nap=5.):
    """Get the reference list or citation list from ADS."""
    wait_for_response = 10. #sec
    try:
        t0 = time.time()
        options = Options()
        options.add_argument(f'user-agent='+random.choice(user_agent_list))
        driver = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/local/bin/chromedriver")
        #driver = webdriver.Chrome(executable_path=r"/usr/local/bin/chromedriver")
        driver.get(url)
        element = WebDriverWait(driver, 50).until(
            EC.presence_of_element_located((By.CLASS_NAME, 's-results-title'))
            )
        #el_per_page = driver.find_element_by_id("per-page-select")
        #el_per_page_500 = el_per_page.find_element_by_xpath("//select[@name='per-page-select']/option[text()='500']")
        #el_per_page_500.click()
        ## Wait for respond
        #time.sleep(wait_for_response)
        source_code = driver.page_source
        soup = BeautifulSoup(source_code)

        bibcode_list_sp = soup.findAll("a", {"aria-label":"bibcode"})
        # Maximum is 500
        page_loop = False
        if (len(bibcode_list_sp) >= 25):
            #print("Warning: this page has at least 500 papers, which means the list could be larger than that.")
            page_loop = True

        print("The number of current page is ", len(bibcode_list_sp))  

        paper_list = []
        count = 0
        for bb in bibcode_list_sp:
            tt = time.time()
            paper1 = PaperExt(ads_id=bb.get_text().strip(), nap=pl_nap(tt-t0, nap))
            paper_list.append(paper1)
            print(count, paper1.title)
            count += 1
        
        count_page = 0
        while(page_loop):
            count_page += 1
            print("Page read ", count_page)
            np_button = driver.find_element(By.XPATH, "//li/a[@class='page-control next-page']")
            try:
                np_button.click()
            except WebDriverException:
                break
            time.sleep(wait_for_response)
            source_code = driver.page_source
            soup = BeautifulSoup(source_code)
            bibcode_list_sp = soup.findAll("a", {"aria-label":"bibcode"})
            print("The number of current page is ", len(bibcode_list_sp))  
            for bb in bibcode_list_sp:
                tt = time.time()
                paper1 = PaperExt(ads_id=bb.get_text().strip(), nap=pl_nap(tt-t0, nap))
                paper_list.append(paper1)
                print(count, paper1.title)
                count += 1
            if (count_page>40):
                print("Warning: Pages exceed 40. Break.")
                break
    except:
        raise
    finally:
        driver.quit()

    print("Sleep...")
    time.sleep(random.random()*nap)
    print("Awake...")
    return ListPapers(paper_list)

In [10]:
def pl_nap(time, nap_min):
    """power law nap time"""
    if(time<nap_min):
        return nap_min
    a = 2.
    b = 0.5
    nap_max = 3600 #1 hour
    return (a*time**b)%nap_max

In [ ]:
test_lp = ads_get_ref_cit("https://ui.adsabs.harvard.edu/abs/2005ApJ...635..931B/citations")
test_lp.summary()

/Users/gaoyuan/programs/anaconda/envs/web3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


The number of current page is  25
0 The prevalence of pseudo-bulges in the Auriga simulations
1 From the outside looking in: what can Milky Way analogues tell us about the star formation rate of our own galaxy?
2 Elemental Abundances in M31: First Alpha and Iron Abundance Measurements in M31's Giant Stellar Stream
3 Two major accretion epochs in M31 from two distinct populations of globular clusters
4 Dragonfly Imaging of the Galaxy NGC 5907: A Different View of the Iconic Stellar Stream
5 Probing the merger history of red early-type galaxies with their faint stellar substructures
6 Under the FIRElight: Stellar Tracers of the Local Dark Matter Velocity Distribution in the Milky Way
7 Multiple retrograde substructures in the Galactic halo: A shattered view of Galactic history
8 Enrichment of Strontium in Dwarf Galaxies
9 Dependence of Galactic Halo Kinematics on the Adopted Galactic Potential
10 The Biggest Splash
11 A Predicted Correlation Between Age Gradient and Star Formation Histor

89 The influence of Sagittarius and the Large Magellanic Cloud on the stellar disc of the Milky Way Galaxy
90 SMHASH: a new mid-infrared RR Lyrae distance determination for the Local Group dwarf spheroidal galaxy Sculptor
91 The First Tidally Disrupted Ultra-faint Dwarf Galaxy?: A Spectroscopic Analysis of the Tucana III Stream
92 The Origin of the 300 km s<SUP>-1</SUP> Stream near Segue 1
93 Galactic cartography with SkyMapper - I. Population substructure and the stellar number density of the inner halo
94 Fitting the Density Substructure of the Stellar Halo with MilkyWay@home
95 Where are the most ancient stars in the Milky Way?
96 Formation and incidence of shell galaxies in the Illustris simulation
97 The extended Planetary Nebula Spectrograph (ePN.S) early-type galaxy survey: The kinematic diversity of stellar halos and the relation between halo transition scale and stellar mass
98 SMHASH: anatomy of the Orphan Stream using RR Lyrae stars
99 Stellar halos in Illustris: probing the

In [12]:
with open("tmp_list.pkl", "wb") as f:
    pck.dump(test_lp, f)

In [15]:
print(test_lp.list_paper[0].doi)

10.1007/BFb0030936
